In [1]:
import os
import mobie
from bioio import BioImage
import bioio_tifffile
from bioio.writers import OmeTiffWriter
import imageio
import shutil

In [2]:
def splitchannels(img,path,dimensions):
    num_channels = len(img.channel_names)
    all = []
    allnames = []
    fname = os.path.basename(path).split(".")[0]

    print(f"Converting {path} to {num_channels} single channel image(s): ")
    for i in range(num_channels):
        if os.path.isfile(f"data/tmp_{fname}_C{i+1}.tif"):
            print(f"\tConverted channel {img.channel_names[i]} :D")
            all.append(f"data/tmp_{fname}_C{i+1}.tif")
            allnames.append(f"{i}_{fname}")
            continue

        try:
            tempimg = img.get_image_data("TCZYX",C=i)
            print(f"\tConverted channel {img.channel_names[i]} :D")
            OmeTiffWriter.save(tempimg, f"data/tmp_{fname}_C{i+1}.tif")
            all.append(f"data/tmp_{fname}_C{i+1}.tif")
            allnames.append(f"{i}_{fname}")
        except:
            print(f"Could not convert channel {img.channel_names[i]} :(")

    return all, allnames, dimensions

In [3]:
def checkchannels(path):
    img = BioImage(path, reader=bioio_tifffile.Reader)
    dimensions = 0

    for i in img.shape:
        if i > 1:
            dimensions += 1

    if len(img.channel_names) > 1:
        return splitchannels(img,path,dimensions)
    else:
        print(f"Only one channel found in {path} - no conversion needed :)")
        return [path], [os.path.basename(path)], dimensions

In [4]:
def addimg(input_path, im_name, menu_name, project_folder, dataset_name, target, max_jobs, dimension, color=None, unit="nanometer"):
    resolution = (10, 10, 10)
    
    # TO-DO: check nD dimensions
    if dimension == 2:
        chunks = (512,512)
        scale_factors = 4 * [[1, 2]]
    else:
        chunks = (64,64,64)
        scale_factors = 4 * [[1, 2, 2]]

    if color is not None:
        view = mobie.metadata.get_default_view("image", im_name, color=color)
    else:
        view = mobie.metadata.get_default_view("image", im_name)

    mobie.add_image(
        input_path=input_path,
        input_key='',
        root=project_folder,
        dataset_name=dataset_name,
        image_name=im_name,
        menu_name=menu_name,
        resolution=resolution,
        chunks=chunks,
        scale_factors=scale_factors,
        is_default_dataset=False,
        target=target,
        max_jobs=max_jobs,
        unit=unit,
        view=view 
    )

In [5]:
def addimgtransform(input_path, im_name, menu_name, project_folder, dataset_name, target, max_jobs, dimension, transform, color=None, unit="nanometer"):
    resolution = (10, 10, 10)
    
    # TO-DO: check nD dimensions
    if dimension == 2:
        chunks = (512,512)
        scale_factors = 4 * [[1, 2]]
    else:
        chunks = (32, 128, 128)
        scale_factors = 4 * [[1, 2, 2]]

    im = imageio.volread(input_path)
    min_val, max_val = im.min(), im.max()
    view = mobie.metadata.get_default_view(
        "image", im_name,
        source_transform={"parameters": transform},
        contrastLimits=[min_val, max_val]
    )

    mobie.add_image(
        input_path=input_path,
        input_key='',
        root=project_folder,
        dataset_name=dataset_name,
        image_name=im_name,
        menu_name=menu_name,
        resolution=resolution,
        scale_factors=scale_factors,
        transformation=transform,
        chunks=chunks,
        target=target,
        max_jobs=max_jobs,
        view=view,
        unit=unit,
        file_format="bdv.n5"
    )

In [6]:
project_folder = "test_project/data"
dataset_name = "example"
dataset_folder = os.path.join(project_folder, dataset_name)
target = "local"
max_jobs = 4
unit = "nanometers"

source_list = []

In [7]:
def add_file(path, menu_name, unit, transform=None, color=None):
    input_file = path
    raw_name = os.path.basename(input_file).split(".")[0]
    menu_name = menu_name

    all, names, dimensions = checkchannels(input_file)

    if transform:
        if len(all) == 1:
            addimgtransform(all[0],raw_name,menu_name, project_folder, dataset_name, target, max_jobs, dimensions, transform, color=color, unit=unit)
            names = [raw_name]
        else:
            for i, x in enumerate(all):
                addimgtransform(x,f"{i}_{raw_name}",menu_name, project_folder, dataset_name, target, max_jobs, dimensions, transform, color=color, unit=unit)

        source_list.append(names)
        return

    if(len(all) == 1):
        addimg(all[0],raw_name,menu_name, project_folder, dataset_name, target, max_jobs, dimensions, color=color, unit=unit)
        names = [raw_name]
    else:
        for i, x in enumerate(all):
            addimg(x,f"{i}_{raw_name}",menu_name, project_folder, dataset_name, target, max_jobs, dimensions, color=color, unit=unit)

    source_list.append(names)

In [8]:
add_file("data/em_20nm_z_40_145.tif", "em", unit)
transformation = [1.8369913620359506,
              -0.5565781005969193,
              -24.28254860249842,
              68.37265958995685,
              -0.08026654420496038,
              1.8421093883743114,
              2.937347223020879,
              -137.17914808990668,
              -6.591949208711867E-17,
              2.5724757460004824E-33,
              0.999999999999993,
              7.638334409421077E-14]
add_file("data/EM04468_2_63x_pos8T_LM_raw.tif", "lm", unit, transform=transformation, color="green")

/Users/najmie/miniconda3/envs/testmobie2/lib/python3.11/site-packages/luigi/parameter.py:296: UserWarning: Parameter "dtype" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
DEBUG: Checking if DownscalingWorkflow(tmp_folder=tmp_example_em_20nm_z_40_145, max_jobs=4, config_dir=tmp_example_em_20nm_z_40_145/configs, target=local, dependency=DummyTask, input_path=data/em_20nm_z_40_145.tif, input_key=, scale_factors=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], halos=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], dtype=None, int_to_uint=False, metadata_format=ome.zarr, metadata_dict={"resolution": [10, 10, 10], "unit": "nanometers", "setup_name": "em_20nm_z_40_145"}, output_path=test_project/data/example/images/ome-zarr/em_20nm_z_40_145.ome.zarr, output_key_prefix=, force_copy=False, skip_existing_levels=False, scale_offset=0) is complete
/Users/najmie/miniconda3/envs/testmobie2/lib/python3.11/si

Only one channel found in data/em_20nm_z_40_145.tif - no conversion needed :)


INFO: [pid 62023] Worker Worker(salt=9963803687, workers=1, host=CH7H1X6L5F, username=najmie, pid=62023) done      CopyVolumeLocal(tmp_folder=tmp_example_em_20nm_z_40_145, max_jobs=4, config_dir=tmp_example_em_20nm_z_40_145/configs, input_path=data/em_20nm_z_40_145.tif, input_key=, output_path=test_project/data/example/images/ome-zarr/em_20nm_z_40_145.ome.zarr, output_key=s0, prefix=initial_scale, dtype=None, int_to_uint=False, fit_to_roi=False, effective_scale_factor=[], dimension_separator=/, dependency=DummyTask)
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   CopyVolumeLocal_tmp_example_em_2_DummyTask___65fe5f142d   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 6
INFO: [pid 62023] Worker Worker(salt=9963803687, workers=1, host=CH7H1X6L5F, username=najmie, pid=62023) running   DownscalingLocal(tmp_folder=tmp_example_em_20nm_z_40_145, max_jobs=4, config_dir=tmp_example_em_20nm_z_40_145/configs, input_path=test

Converting data/EM04468_2_63x_pos8T_LM_raw.tif to 4 single channel image(s): 
	Converted channel Channel:0:0 :D
	Converted channel Channel:0:1 :D
	Converted channel Channel:0:2 :D
	Converted channel Channel:0:3 :D


/Users/najmie/miniconda3/envs/testmobie2/lib/python3.11/site-packages/luigi/parameter.py:296: UserWarning: Parameter "dtype" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
DEBUG: Checking if DownscalingWorkflow(tmp_folder=tmp_example_0_EM04468_2_63x_pos8T_LM_raw, max_jobs=4, config_dir=tmp_example_0_EM04468_2_63x_pos8T_LM_raw/configs, target=local, dependency=DummyTask, input_path=data/tmp_EM04468_2_63x_pos8T_LM_raw_C1.tif, input_key=, scale_factors=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], halos=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], dtype=None, int_to_uint=False, metadata_format=bdv.n5, metadata_dict={"resolution": [10, 10, 10], "unit": "nanometers", "setup_name": "0_EM04468_2_63x_pos8T_LM_raw"}, output_path=test_project/data/example/images/bdv-n5/0_EM04468_2_63x_pos8T_LM_raw.n5, output_key_prefix=, force_copy=False, skip_existing_levels=False, scale_offset=0) is complete
/Us

In [9]:
settings = [{"color": "white", "contrastLimits": [0., 255.]}, {"opacity": 0.75}]

In [10]:
mobie.create_view(dataset_folder, "default",
                  sources=source_list, display_settings=settings,
                  overwrite=True)

/Users/najmie/miniconda3/envs/testmobie2/lib/python3.11/site-packages/mobie/metadata/dataset_metadata.py:72: UserWarning: A view with name default already exists for the dataset test_project/data/example
  warnings.warn(msg)


In [11]:
for dir in os.listdir(os.getcwd()):
    if(dir[0:3] == "tmp"):
        shutil.rmtree(dir)

for file in os.listdir("data"):
    if(file[0:3] == "tmp"):
        os.remove(os.path.join("data",file))